## Julia Execution Time in Comparison

Code was simplified to only estimate $\pi$ instead of storing all simulated points

In [3]:
using BenchmarkTools
using PyCall

In [4]:
function bench_monte_carlo_pi_jl(n_sim_points::Int64)
    n_in::Int64 = 0
    for i in 0:n_sim_points
        # create two random numbers for x, y
        x = rand() - 0.5
        y = rand() - 0.5
        # if the norm is less then the radius, add to point n_outside
        if x^2 + y^2 <= 0.25
           n_in += 1 
        end 
    end
    return 4*n_in/n_sim_points
end

bench_monte_carlo_pi_jl (generic function with 1 method)

In [5]:
n_t = 1_000_000
@benchmark bench_monte_carlo_pi_jl(n_t)

BenchmarkTools.Trial: 2231 samples with 1 evaluation.
 Range (min … max):  2.218 ms …  2.629 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.233 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.241 ms ± 31.006 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▇█▆▆▆▄▄▃▃▁▁                                              ▁
  ▆██████████████▆▇▇▆▆▇▁▆▅▆▆▇▅▆▄▄▄▄▇▆▁▁▁▄▆▁▄▅▅▄▁▁▄▁▁▁▄▄▄▄▁▁▄ █
  2.22 ms      Histogram: log(frequency) by time      2.4 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

## Python
Python execution is directly possible within Julia.

In [6]:
py"""
import random, math
def bench_monte_carlo_pi_py(n_sim_points):
    n_in = 0
    for i in range(0,n_sim_points):
        x = random.uniform(-0.5, 0.5)
        y = random.uniform(-0.5, 0.5)
        if x**2 + y**2 <= 0.25:
            n_in +=1
    return 4*n_in/n_sim_points
"""

In [7]:
@benchmark py"bench_monte_carlo_pi_py"(n_t)

BenchmarkTools.Trial: 16 samples with 1 evaluation.
 Range (min … max):  315.600 ms … 333.791 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     319.170 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   320.513 ms ±   4.907 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▁▁ ▁ ▁ ▁▁    ▁▁ ▁          █▁    ▁                         ▁  
  ███▁█▁█▁██▁▁▁▁██▁█▁▁▁▁▁▁▁▁▁▁██▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  316 ms           Histogram: frequency by time          334 ms <

 Memory estimate: 192 bytes, allocs estimate: 6.

## C/C++

Julia is able to load dynamically linked C/C++ libraries. This snipped compiles the C++ code with `clang` to a shared library and
loads the library with `@ccall`

In [8]:
using Libdl
c_code = """
#include <random>

extern "C" double bench_monte_carlo_pi_c(int64_t n_sim_points) {
  srand (time(NULL));
  int64_t n_in = 0;
  auto gen = std::default_random_engine();
  auto distribution = std::uniform_real_distribution<double>(-0.5, 0.5);
  
  for (size_t i = 0; i < n_sim_points; i++) {
    double x = distribution(gen);
    double y = distribution(gen);
    if (x * x + y * y <= 0.25) {
      n_in++;
    }
  }
  return 4 * double(n_in) / n_sim_points;
}
"""

const Clib = tempname()
open(`clang -shared -x c++ -O3 -std=c++20 -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code)
end

In [9]:
@benchmark @ccall Clib.bench_monte_carlo_pi_c(n_t::Int64)::Float64

BenchmarkTools.Trial: 332 samples with 1 evaluation.
 Range (min … max):  14.848 ms …  15.877 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     14.937 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.073 ms ± 260.453 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁▅█▇▅▃▂ ▂▁  ▁ ▁ ▁                                             
  ██████████▇██▇█▇█▆▆█▇▇▆▁▄█▆▇▄▇▇▇▆▇▇▆▄▄▄▁▇▄▄▁▆▆▇▆▄▆▆▄▁▇▆▄▆▆▁▆ ▇
  14.8 ms       Histogram: log(frequency) by time      15.8 ms <

 Memory estimate: 16 bytes, allocs estimate: 1.